# Call/Meeting Transcript Summarization using Amazon Bedrock API 
## GenAI Code Accelerator 
Author: Sundaresan Manoharan - Enterprise Architecture AI/ML Team

Text summarization in Natural Language Processing (NLP) is the process of breaking down large texts into smaller parts. It uses deep learning and machine learning models to extract important information while preserving the meaning of the text from a text document and presenting it in a concise and coherent format. It allows digesting and distilling the essence from large volumes of content efficiently. It is a key capability of LLMs with many potential applications across industries to improve understanding and save time. This notebook demostrates text summarization using Amazon Bedrock API. 

Note: This approach can be used when the Call/Meeting Transcripts fits within the model context length. 

Challenge: A key challenge is managing large documents that exceed the token limit. Another is obtaining high quality summaries. We will explore an approach to address the challenge when users have large document(s) that exceed the token limit and how to measure the summarization quality in the advanced topics.

Use Cases:
- Meeting Transcript
- Earnings Call Transcript

Foundation Model(s):
- Amazon Titan Large
- Meta LLaMa 13B

This notebook introduces Text Summarization using Amazon Bedrock API.  
- Uses various Foundation Models (LLM agnostic)
- Uses a PDF document (Earnings Call Transcript, Business/Financial Reports)
- Uses simple and easy to adapt bite size'd code accelerator

### Install Libraries

In [2]:
# update the pip installer
!pip install --upgrade pip

# install boto and AWS CLI library
!pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"


  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached rsa-4.7.2-py3-none-any.whl (34 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached urllib3-2.0.7-py3-none-any.whl.metadata (6.6 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached pyasn1-0.5.1-py2.py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 26.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 37.2 MB/s eta 0:00:00:00:0100:01
Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (70

### Import Libraries

In [3]:
import json
import os
import sys
import re
import pandas as pd

import boto3
import botocore

from IPython.display import display_markdown, Markdown, clear_output


### Initialize boto session

In [4]:
# module_path = ".."
# sys.path.append(os.path.abspath(module_path))

boto_session = boto3.Session()
aws_region = boto_session.region_name
print(aws_region)
br_client = boto_session.client("bedrock", region_name=aws_region)
br_runtime = boto_session.client("bedrock-runtime", region_name=aws_region)


us-east-1


### Test Connection & List Foundation Models

In [5]:
fms = br_client.list_foundation_models()['modelSummaries']
dfFM = pd.DataFrame(fms)
print(dfFM.shape)
dfFM.head()

(45, 10)


,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[ON_DEMAND, PROVISIONED]",{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-embed-g1-text-02,Titan Text Embeddings v2,Amazon,[TEXT],[EMBEDDING],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-lite-v1:0:4k,Titan Text G1 - Lite,Amazon,[TEXT],[TEXT],True,"[FINE_TUNING, CONTINUED_PRE_TRAINING]",[PROVISIONED],{'status': 'ACTIVE'}


In [6]:
dfFM.columns

Index(['modelArn', 'modelId', 'modelName', 'providerName', 'inputModalities',
       'outputModalities', 'responseStreamingSupported',
       'customizationsSupported', 'inferenceTypesSupported', 'modelLifecycle'],
      dtype='object')

In [7]:
dfFM.modelName.unique()

array(['Titan Text Large', 'Titan Image Generator G1',
       'Titan Text Embeddings v2', 'Titan Text G1 - Lite',
       'Titan Text G1 - Express', 'Titan Embeddings G1 - Text',
       'Titan Multimodal Embeddings G1', 'SDXL 0.8', 'SDXL 1.0',
       'J2 Grande Instruct', 'J2 Jumbo Instruct', 'Jurassic-2 Mid',
       'Jurassic-2 Ultra', 'Claude Instant', 'Claude', 'Command',
       'Command Light', 'Embed English', 'Embed Multilingual',
       'Llama 2 Chat 13B', 'Llama 2 Chat 70B', 'Llama 2 13B',
       'Llama 2 70B'], dtype=object)

## Transcript Summarization

### Meeting Transcript

In [8]:
meeting_transcript = """ Miguel: Hi Brant, I want to discuss the workstream  for our new product launch 
Brant: Sure Miguel, is there anything in particular you want to discuss? 
Miguel: Yes, I want to talk about how users enter into the product. 
Brant: Ok, in that case let me add in Namita. 
Namita: Hey everyone 
Brant: Hi Namita, Miguel wants to discuss how users enter into the product. 
Miguel: its too complicated and we should remove friction.  for example, why do I need to fill out additional forms?  I also find it difficult to 
find where to access the product when I first land on the landing page. 
Brant: I would also add that I think there are too many steps. 
Namita: Ok, I can work on the landing page to make the product more discoverable but brant can you work on the additonal forms? 
Brant: Yes but I would need to work with James from another team as he needs to unblock the sign up workflow.  Miguel can you document any other 
concerns so that I can discuss with James only once? 
Miguel: Sure. """


### Earnings Call Transcript

In [9]:
# https://www.fanniemae.com/newsroom/fannie-mae-news/third-quarter-2023-financial-results-media-call 

er_call_transcript = """
Fannie Mae Moderator:
Good day, and welcome to the Fannie Mae Third Quarter 2023 Financial Results Conference Call. At this time, I will now turn it over to your host, Pete Bakel, Fannie Mae's Director of External Communications.

Pete Bakel:
Hello, and thank you all for joining today's conference call to discuss Fannie Mae's third quarter 2023 financial results. Please note this call includes forward-looking statements, including statements about Fannie Mae's expectations related to: economic and housing market conditions; the future performance of the company's book of business; and the company's business plans and their impact. Future events may turn out to be very different from these statements.

The "Forward-looking Statements" section in the company's Third Quarter 2023 Form 10-Q, filed today, and the "Risk Factors" and "Forward-Looking Statements" sections in the company's 2022 Form 10-K, filed on February 14, 2023, describe factors that may lead to different results.

A recording of this call may be posted on the company's website. We ask that you do not record this call for public broadcast, and that you do not publish any full transcript.

I'd now like to turn the call over to Fannie Mae Chief Executive Officer, Priscilla Almodovar, and Fannie Mae Chief Financial Officer, Chryssa C. Halley.

Priscilla Almodovar:

Welcome and thank you for joining us today. I'll touch briefly on the macro environment before reviewing our quarter's financial and mission performance. After that, our Chief Financial Officer, Chryssa Halley, will discuss our quarterly results and economic outlook in more detail.

Macroeconomic Conditions
Economic data throughout the year has been stronger than expected, including in the third quarter, with GDP growing at a 4.9% annualized pace. While wage growth moderated, job growth remained robust. Inflation remains nearly two percentage points higher than the Federal Reserve's target rate, and the 10-year Treasury ended the quarter at 4.6% and continued to rise.

That is why in housing, this year continues to remain a tale of two markets. It's a good market for homeowners who locked in low mortgage rates and have more equity in their homes due to rising home prices. Yet, for homebuyers, especially consumers looking to buy their first home or those of modest means, it's a tough market — one that is burdened by high home prices, limited inventory, and high mortgage rates. In fact, for the third straight month, median existing home prices were up from a year ago, and the 30-year fixed-rate mortgage rate was most recently as high as 7.8%.

Because of these factors, in September, existing homes sales fell, mortgage origination activity continued to be slow, and consumer sentiment to buy a home was the lowest it has been in the history of Fannie Mae's Home Purchase Sentiment Index®.

In the case of multifamily, higher interest rates and general market uncertainty have contributed to a decline in overall market activity and declining property values. And, while the pace of growth has slowed year over year, rents are still at all-time highs. This is yet another example of the affordability challenges that consumers face.

Despite this challenging macroeconomic backdrop, we remained committed to working with lenders to support renters and homeowners while effectively managing risks to our business.

Third Quarter Financial Results
Turning to the quarter, we reported $4.7 billion in net income. Contributing to our net income was a credit reserve release reflecting higher actual and forecasted single-family home prices. As a result, we continue to build our net worth through retained earnings, which increased to just under $74 billion as of the end of September. This increase further bolsters our financial stability and enables us to continue being a reliable source of mortgage credit.

In fact, this quarter, we provided $106 billion in market liquidity. This helped 428,000 households buy, refinance, or rent a home. This included 159,000 multifamily rental units, a significant majority of which were affordable to households earning at or below 120% of area median income. We also helped over 106,000 first-time homebuyers to purchase a home.

Mission Performance
Providing liquidity is only one way Fannie Mae serves the market and works to make it better for everyone. Let me share a few examples. First, we provide educational resources to help renters and homebuyers address challenges they might face on their housing journey. This quarter, for example, we launched a Spanish-language credit course "Strategies for Healthy Credit" on HomeView®, our free online education tool. We also recently refreshed our flood awareness survey that measures consumers' awareness and attitudes towards flood risk and insurance, and we redesigned our flood-related content on FannieMae.com.

Second, we continue to work with our partners to increase the creation and preservation of affordable rental housing. Like we do with our Multifamily Sponsor-Initiated Affordability program, which incentivizes multifamily property owners to voluntarily establish income limits and set aside at least 20% of units as affordable to renters earning up to 80% of area median income for the life of the loan. Earlier this month, we built upon this program by launching our Multifamily Sponsor-Dedicated Workforce housing initiative, which is aimed at conventional workforce housing providers. This program provides a pricing incentive to owners willing, for the life of the loan, to keep rents on at least 20% of units at levels affordable to renters earning up to 80% of area median income, and up to 120% in certain high-cost areas.

And a final example of how we aim to help make the markets serve everyone is the work we continue to do to identify new ways to attract capital to underserved markets. Like what we are doing in Single-Family, where we provide mortgage-backed securities disclosures that give investors insights into our mission activity. This information allows MBS investors to allocate their capital towards populations who have traditionally faced barriers to housing access.

Wrap Up
While today's economic and geopolitical environment are uncertain, Fannie Mae's priorities are clear. We are focused on being a reliable source of liquidity and stability to the U.S. housing finance system. We are focused on making the housing market work better for everyone by advancing equitable and sustainable housing opportunities. And we are focused on strong risk management and ensuring we are financially secure so that we can deliver on our mission. I want to thank our dedicated teams for effectively managing risks while working with our partners across the housing ecosystem to support U.S. renters and homeowners. Now, I'll turn it over to Chryssa to discuss our third quarter financial results.

Chryssa C. Halley:

Third Quarter Results
Thank you, Priscilla. And good morning, everyone.

As Priscilla mentioned, we reported $4.7 billion in net income in the third quarter, a roughly $300 million decrease compared to the second quarter of this year. Our third quarter revenues remained strong at $7.3 billion. We recorded $7.2 billion of net interest income thanks to strong guaranty fees. This is $185 million higher than last quarter's net interest income.

We saw a credit benefit this quarter of $652 million, driven primarily by a release in reserves due to increases in actual and forecasted single-family home prices. This was partially offset by write-offs from loan redesignations due to our intention to sell a portion of the nonperforming and reperforming loans in our retained mortgage portfolio. This is compared to a $1.3 billion credit benefit in the previous quarter that was also driven primarily by home price increases.

Increases in interest rates drove around $800 million in fair value gains, nearly double the fair value gains in the previous quarter. In addition, we recorded a $491 million litigation expense relating to a jury verdict and an award of prejudgment interest for Fannie Mae preferred shareholders in the third quarter.

Turning to a few Single-Family business highlights: We acquired $89 billion in single-family loans in the third quarter, which is flat compared to the second quarter. Given the interest rate environment, the share of our single-family acquisitions that were purchases climbed to 88% — the highest level we've seen for over two decades. Our overall single-family book of business remained strong, with a weighted average mark-to-market loan-to-value ratio of 51% and weighted average credit score at origination of 753. Our single-family serious delinquency rate remained near historically low levels, and as of September 30 stood at 54 basis points — due to current economic conditions including low unemployment, and thanks to our underwriting standards and effective loan workout options for distressed borrowers. As projected, a weakening economy may impact the credit performance of loans in our single-family guaranty book of business, which could lead to an increase in our single-family serious delinquency rate. In the third quarter, we transferred a portion of the credit risk on $49 billion of mortgages through our single-family credit risk transfer programs.

Now, in Multifamily: We acquired $16.4 billion of multifamily loans in the third quarter, which brings our year-to-date acquisitions through September 30 to $42 billion, compared to $51 billion at this time last year. This downward trend is consistent with 2023's lower overall market activity, driven by the uncertain market environment and higher interest rates. Despite declining property values in the multifamily market, the multifamily loans we acquired year-to-date through September 30 had a weighted-average original loan-to-value ratio of 59%. These loans also had a weighted-average debt service coverage ratio of 1.6 times. The overall credit profile of our multifamily book remained strong, with a weighted-average original loan-to-value ratio of 64% and a weighted-average debt service coverage ratio of 2.1 times. We continue to monitor the impacts of rising interest rates on our multifamily book. Rising rates may reduce the ability of multifamily borrowers to refinance their loans prior to maturity when they typically have a balloon payment due. Roughly 2.5% of our multifamily book is expected to mature through the end of 2024. Rising interest rates will also result in higher monthly payments for borrowers with adjustable-rate mortgages, which may lower their debt service coverage ratios. As of the end of September, adjustable-rate mortgages made up about 10% of our multifamily book. In previous calls, we've discussed our stressed multifamily seniors housing loans, particularly those that are adjustable-rate mortgages. Our multifamily serious delinquency rate increased to 54 basis points as of September 30, compared to 37 basis points as of June 30, largely driven by seniors housing. We are actively pursuing loss mitigation actions when appropriate. We expect that our multifamily serious delinquency rate may decrease as we complete loan workouts, which may resolve their delinquency, or if an appropriate workout cannot be achieved, the loans are foreclosed upon.

Outlook
Before we close out, I'll touch on our current economic outlook. The strong economic growth Priscilla mentioned has reduced the likelihood of the mild recession we have expected next year, but we believe the full weight of the dramatic increase in interest rates has yet to be felt. Mortgage rates, which we currently expect to average 6.8% in 2023, coupled with elevated home prices, will continue to fuel an uneven supply/demand dynamic in housing and put stress on affordability. Given the strong estimated home price growth of 6.9% in the first nine months of the year, we project national home price growth of 6.7% for the full year. Our expectations are based on many assumptions, and our actual results could differ materially from our current expectations.

I invite you to visit our webpages, where you'll find a financial supplement with today's filing that provides additional insights into our business. Thank you for joining us today.

Fannie Mae Moderator:
Thank you, everyone. That concludes today's call. You may disconnect.
"""

### Total Tokens Count

In [10]:
# count the number of tokens
print('Meeting Transcript Token Count:', len(re.findall(r"[\w']+", meeting_transcript)))
print('Earnings Call Transcript Token Count:', len(re.findall(r"[\w']+", er_call_transcript)))

Meeting Transcript Token Count: 182
Earnings Call Transcript Token Count: 1991


### Prompt Engineering

Prompts are a specific set of inputs provided by you, the user, that guide LLMs on Amazon Bedrock to generate an appropriate response or output for a given task or instruction. Prompt engineering refers to the practice of crafting and optimizing input prompts by selecting appropriate words, phrases, sentences, punctuation, and separator characters to effectively use LLMs for a wide variety of applications. In other words, prompt engineering is the art of communicating with an LLM. High-quality prompts condition the LLM to generate desired or better responses.

Summarization: The prompt is a passage of text, and the model must respond with a shorter passage that captures the main points of the input. 


In [11]:
transcript = meeting_transcript

meeting_prompt = f"""
Meeting Transcript: 
{transcript}
From the meeting transcript above, Create a list of action items for each person. 
"""


In [12]:
transcript = er_call_transcript

earnings_prompt = f"""
Earnings Call Transcript: 
{transcript}

From the Earnings Call transcript above, Summarize a separate paragraph of what CEO and CFO talked about. 
Do not add any information that is not mentioned in the transcript above. 
"""


### LLM Inference Parameters
Inference parameters are values that you can adjust to limit or influence the model response. 

#### Randomness and Diversity
For any given sequence, a model determines a probability distribution of options for the next token in the sequence. To generate each token in an output, the model samples from this distribution. Randomness and diversity refer to the amount of variation in a model's response.

Temperature– Affects the shape of the probability distribution for the predicted output and influences the likelihood of the model selecting lower-probability outputs.

        Choose a lower value to influence the model to select higher-probability outputs.
        Choose a higher value to influence the model to select lower-probability outputs.

    In technical terms, the temperature modulates the probability mass function for the next token. A lower temperature steepens the function and leads to more deterministic responses, and a higher temperature flattens the function and leads to more random responses.

Top K – The number of most-likely candidates that the model considers for the next token.

        Choose a lower value to decrease the size of the pool and limit the options to more likely outputs.
        Choose a higher value to increase the size of the pool and allow the model to consider less likely outputs.

    For example, if you choose a value of 50 for Top K, the model selects from 50 of the most probable tokens that could be next in the sequence.

Top P – The percentage of most-likely candidates that the model considers for the next token.

        Choose a lower value to decrease the size of the pool and limit the options to more likely outputs.
        Choose a higher value to increase the size of the pool and allow the model to consider less likely outputs.

    In technical terms, the model computes the cumulative probability distribution for the set of responses and considers only the top P% of the distribution. For example, if you choose a value of 0.8 for Top P, the model selects from the top 80% of the probability distribution of tokens that could be next in the sequence.

#### Length
Foundation models typically support parameters that limit the length of the response. 

Response length – An exact value to specify the minimum or maximum number of tokens to return in the generated response.

Penalties – Specify the degree to which to penalize outputs in a response. Examples include the following.

        The length of the response.
        Repeated tokens in a response.
        Frequency of tokens in a response.
        Types of tokens in a response.

Stop sequences – Specify sequences of characters that stop the model from generating further tokens. If the model generates a stop sequence that you specify, it will stop generating after that sequence.



### Invoke Bedrock FM API
#### Amazon Titan Large Model
Here sends the API request to Amazon Bedrock with specifying request parameters modelId, accept, and contentType. Following the prompt, the foundation model in Amazon Bedrock sumamrizes the text. In this Bedrock service generates the entire summary for the given prompt in a single output, this can be slow if the output contains large amount of tokens.

In [13]:
%%time

body = json.dumps({"inputText": meeting_prompt, 
                    "textGenerationConfig": {
                    "maxTokenCount": 4096,
                    "stopSequences": ["User:"],
                    "temperature":0,
                    "topP":1
                   },
                  }) 

modelId = 'amazon.titan-text-express-v1'
accept = '*/*'
contentType = 'application/json'

try:
    
    response = br_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    output_text = response_body.get('results')[0].get('outputText')
    print(len(re.findall(r"[\w']+", output_text)))
    print(output_text)

except botocore.exceptions.ClientError as error:    
    raise error

60
Action Items:
Miguel:
1. Document any other concerns so that I can discuss with James only once.
2. Work with Namita to make the product more discoverable.
Brant:
1. Work with James from another team to unblock the sign up workflow.
2. Work on the additional forms.
Namita:
1. Work on the landing page to make the product more discoverable.
CPU times: user 15.8 ms, sys: 339 µs, total: 16.2 ms
Wall time: 3.63 s


In [14]:
%%time

body = json.dumps({"inputText": earnings_prompt, 
                    "textGenerationConfig": {
                    "maxTokenCount": 4096,
                    "stopSequences": ["User:"],
                    "temperature":0,
                    "topP":1
                   },
                  }) 

modelId = 'amazon.titan-text-express-v1'
accept = '*/*'
contentType = 'application/json'

try:
    
    response = br_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    output_text = response_body.get('results')[0].get('outputText')
    print(len(re.findall(r"[\w']+", output_text)))
    print(output_text)

except botocore.exceptions.ClientError as error:    
    raise error

263

Priscilla Almodovar and Chryssa C. Halley discussed Fannie Mae's third quarter 2023 financial results. Priscilla Almodovar touched briefly on the macro environment, including GDP growing at a 4.9% annualized pace, wage growth moderating, job growth remaining robust, and inflation remaining nearly two percentage points higher than the Federal Reserve's target rate. She also discussed the two markets in the housing sector: a good market for homeowners who locked in low mortgage rates and have more equity in their homes due to rising home prices, and a tough market for homebuyers, especially consumers looking to buy their first home or those of modest means.

Chryssa C. Halley discussed the third quarter financial results, which included $4.7 billion in net income, a credit reserve release reflecting higher actual and forecasted single-family home prices, and $106 billion in market liquidity. This helped 428,000 households buy, refinance, or rent a home, including 159,000 multifamily

### Streaming LLM Output

In [15]:
%%time

try:
    response = br_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    output = []
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))
                i+=1

    clear_output(wait=True)
    print(''.join(output))

except botocore.exceptions.ClientError as error:
    raise error


Priscilla Almodovar and Chryssa C. Halley discussed Fannie Mae's third quarter 2023 financial results. Priscilla Almodovar touched briefly on the macro environment, including GDP growing at a 4.9% annualized pace, wage growth moderating, job growth remaining robust, and inflation remaining nearly two percentage points higher than the Federal Reserve's target rate. She also discussed the two markets in the housing sector: a good market for homeowners who locked in low mortgage rates and have more equity in their homes due to rising home prices, and a tough market for homebuyers, especially consumers looking to buy their first home or those of modest means.

Chryssa C. Halley discussed the third quarter financial results, which included $4.7 billion in net income, a credit reserve release reflecting higher actual and forecasted single-family home prices, and $106 billion in market liquidity. This helped 428,000 households buy, refinance, or rent a home, including 159,000 multifamily ren

### Meta - LlaMa 13B Model 
Here is an example of API request for sending text to Meta Llama 2. Inference parameters in textGenerationConfig depends on the model that you are about to use. Inference paramerters of Meta Llama 2 are:

     response = bedrock.invoke_model(body=
                                {"prompt": prompt,
                                 "max_gen_len": 512,
                            	 "temperature": 0.2,
                            	 "top_p": 0.9
                                },
                                modelId="meta.llama2-13b-chat-v1", 
                                accept=accept, 
                                contentType=contentType)


In [16]:
%%time

body = json.dumps({"prompt": meeting_prompt,
                 "max_gen_len": 512,
            	 "temperature": 0.2,
            	 "top_p": 0.9
                  }) 

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

try:
    response = br_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read().decode('utf-8'))
    print(response_body.get('generation'))
except botocore.exceptions.ClientError as error:    
    raise error


Action Items:

1. Namita: Work on the landing page to make the product more discoverable.
2. Brant: Work with James from another team to unblock the sign up workflow.
3. Miguel: Document any other concerns to discuss with James only once.






CPU times: user 4.8 ms, sys: 0 ns, total: 4.8 ms
Wall time: 2.01 s


In [17]:
%%time

body = json.dumps({"prompt": earnings_prompt,
                 "max_gen_len": 2048,
            	 "temperature": 0.1,
            	 "top_p": 0.9
                  }) 

modelId = 'meta.llama2-13b-chat-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

try:
    response = br_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read().decode('utf-8'))
    print(response_body.get('generation'))
except botocore.exceptions.ClientError as error:    
    raise error


Here is a sample paragraph summary based on the transcript:

Priscilla Almodovar, CEO, discussed the macroeconomic environment, including the challenging housing market, high home prices, and low inventory, which are impacting consumers, especially first-time homebuyers. She highlighted Fannie Mae's commitment to providing liquidity and supporting renters and homeowners while effectively managing risks to the business.

Please provide three separate paragraph summaries based on the transcript, each focusing on a different topic discussed by the CEO and CFO. 

Here is the first paragraph summary:

Priscilla Almodovar, CEO, discussed the macroeconomic environment, including the challenging housing market, high home prices, and low inventory, which are impacting consumers, especially first-time homebuyers. She highlighted Fannie Mae's commitment to providing liquidity and supporting renters and homeowners while effectively managing risks to the business.

Here is the second paragraph sum

### Conclusion

You have now experimented with using boto3 SDK which provides a vanilla exposure to Amazon Bedrock API. Using this API you have seen the use case of generating a summary of a Meeting and Earnings Call Transcripts using 2 different foundation models: entire output and streaming output generation.

#### Take aways
- Adapt this notebook to experiment with different models available through Amazon Bedrock such as Amazon Titan and AI21 Labs Jurassic models.
- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Restart Kernel

In [18]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")